In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext

import numpy as np

# Create a Spark session
spark = SparkSession.builder.appName("MySparkApp").getOrCreate()
sc = SparkContext.getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/08 09:57:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:

def readFile(filename):
    # Initialize Spark context
    #spark = SparkSession.builder.appName("MySparkApp").getOrCreate()

    #sc = spark.sparkContext

    # Read the file into an RDD
    # Each line in the file becomes one record in the RDD
    lines = sc.textFile(filename)

    # Process each line in the RDD
    data = lines.map(lambda line: line.split(','))\
                .map(lambda elem: ([float(x) for x in elem[:-1]], int(elem[-1])))

    return data

In [3]:
#Testing of the read function
filename = 'data/botnet_tot_syn_l.csv'
RDD_Xy = readFile(filename)

# Printing first 5 rows of the RDD
for row in RDD_Xy.take(5):
    print(row)

([9.012784269851089, 1672.9999766891833, 21.99998846107087, 0.9999997452701503, 61.99988768910407, 69.99980788311223, 13.000000232993186, 2.9999999785934133, 199.00000192395984, 2468369573.0148935, 2468372549.224571], 1)
([3599.9990884099398, 48206.57583515873, 13362.999847123127, 1.0000019205086303, 262.99924185046257, 82.99988759205826, 13.999945225127506, 5.000000032871429, 216.99999612378573, 1539044199.5873349, 2468368394.7593513], 0)
([0.0006999386757797765, 0.0009472844685660675, 53.000260213743786, 2.0000004196928174, 646242903.4069201, 82.99988759205826, 12.999999992936175, 2.999999985175615, 216.99999612378573, 2468369544.907854, 2503250078.1888514], 0)
([3599.9990884099398, 1718.7743566535646, 63377.68872854763, 1.0000019205086303, 926682.197305462, 543.5568914038613, 12.999999992936175, 5.000000032871429, 186.99999400676145, 405749204.8178947, 1125424493.3033822], 0)
([3471.3792185025277, 48024.1859162382, 0.00035088937602267833, 507008.9762772573, 262.99924185046257, 1000.

In [4]:
'''
def compute_stats(RDD):
    # Calculate sum and sum of squares for each feature
    sums = RDD.map(lambda x: x[0]).reduce(lambda a, b: [a[i] + b[i] for i in range(len(a))])
    return sums
print(compute_stats(rdd))
'''

'\ndef compute_stats(RDD):\n    # Calculate sum and sum of squares for each feature\n    sums = RDD.map(lambda x: x[0]).reduce(lambda a, b: [a[i] + b[i] for i in range(len(a))])\n    return sums\nprint(compute_stats(rdd))\n'

In [5]:
def compute_mean_stds(RDD):
    # Calculate sum and sum of squares for each feature
    sums = RDD.map(lambda x: x[0]).reduce(lambda a, b: [a[i] + b[i] for i in range(len(a))]) 
    sum_of_squares = RDD.map(lambda x: [xi ** 2 for xi in x[0]]).reduce(lambda a, b: [a[i] + b[i] for i in range(len(a))])
    count = RDD.count()

    # Compute mean and standard deviation for each feature
    means = [sums[i] / count for i in range(len(sums))]
    stds = [(sum_of_squares[i] / count - (means[i]) ** 2) ** 0.5 for i in range(len(means))]
    
    return means, stds



def normalize(RDD_Xy):
    # Compute means and standard deviations
    means, stds = compute_mean_stds(RDD_Xy)

    # Broadcast the means and standard deviations
    bc_means = sc.broadcast(means)
    bc_stds = sc.broadcast(stds)

    # Normalize each feature
    def normalize_features(features, means, stds):
        return [(features[i] - means[i]) / stds[i] if stds[i] != 0 else 0 for i in range(len(features))]
        #return [(features[i] - means[i]) / stds[i] for i in range(len(features))]

    normalized_RDD = RDD_Xy.map(lambda x: (normalize_features(x[0], bc_means.value, bc_stds.value), x[1]))

    bc_means.unpersist()
    bc_stds.unpersist()


    return normalized_RDD



In [6]:
#Testing of the normalize function

normalized_RDD = normalize(RDD_Xy).cache()

# Print first 5 normalized rows
for row in normalized_RDD.take(5):
    print(row)


([-0.792409774778866, -0.8130937083307339, -0.4224507642218906, -0.4664697491193844, -0.5223929554497563, -0.35631956924945285, 0.7370102960327225, 0.5283496333423401, 0.8271779941619601, 0.4731661571170488, 0.15895172357621995], 1)
([1.443720404002351, 1.1163610034798641, 0.39127563577819524, -0.4664697491031832, -0.5223920961870128, -0.35631927462193325, 0.9263319360863808, 1.480120748267978, 1.0264183320400917, -0.8286954523619166, 0.15894853159524694], 0)
([-0.7980216574316705, -0.8824624555329105, -0.4205599209717913, -0.46646230111358483, 2.240264662197529, -0.35631927462193325, 0.7370102505822355, 0.5283496364747148, 1.0264183320400917, 0.4731661177428138, 0.185749013466657], 0)
([1.443720404002351, -0.8111957326975887, 3.441892018491165, -0.4664697491031832, -0.5184316978805608, -0.35630883678072567, 0.7370102505822355, 1.480120748267978, 0.6943509718073647, -2.416291452741655, -0.8728693208325027], 0)
([1.3636279869526706, 1.108798440707592, -0.42379261911074273, 3.30972605187

In [7]:
'''
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def linear_combination(features, weights, bias):
    return sum(weights[j] * features[j] for j in range(len(weights))) + bias

def compute_cost(RDD, w, b, lambda_reg, m):
    
    bc_w = sc.broadcast(w)
    bc_b = sc.broadcast(b)

    total_cost = RDD.map(lambda x: (x[1] * np.log(sigmoid(linear_combination(x[0], bc_w.value, bc_b.value))) + (1 - x[1]) * np.log(1 - sigmoid(linear_combination(x[0], bc_w.value, bc_b.value)))
    )).reduce(lambda a, b: a + b)
    reg_cost = (lambda_reg / (2 * 11)) * np.sum(np.square(w))
    cost = (- total_cost / m) + reg_cost

    #bc_w.unpersist()
    #bc_b.unpersist()
    
    return cost

def compute_gradients(record, weights, bias):
    xi, yi = record
    y_hat = sigmoid(linear_combination(xi, weights, bias))
    error = y_hat - yi
    dw = np.array([error * xi[j] for j in range(len(weights))])
    db = error
    return dw, db

def train(RDD_Xy, iterations, learning_rate, lambda_reg):
    np.random.seed(42) 
    feature_count = len(RDD_Xy.first()[0])
    w = np.random.rand(feature_count)
    b = np.random.rand()
    m = RDD_Xy.count()

    for n in range(iterations):
        bc_w = sc.broadcast(w)
        bc_b = sc.broadcast(b)

        dw, db = RDD_Xy.map(lambda x: compute_gradients(x, bc_w.value, bc_b.value)).reduce(lambda a, b: (np.add(a[0], b[0]), a[1] + b[1]))
        

        # Update weights and bias
        w -= learning_rate * ((dw / m) + lambda_reg * w)
        b -= learning_rate * db / m

        # Compute and print cost
        cost = compute_cost(RDD_Xy, w, b, lambda_reg, m)
        print(f"Iteration {n+1}/{iterations} - Cost: {cost}")

        #bc_w.unpersist()
        #bc_b.unpersist()

    return w, b
'''


'\ndef sigmoid(z):\n    return 1 / (1 + np.exp(-z))\n\ndef linear_combination(features, weights, bias):\n    return sum(weights[j] * features[j] for j in range(len(weights))) + bias\n\ndef compute_cost(RDD, w, b, lambda_reg, m):\n    \n    bc_w = sc.broadcast(w)\n    bc_b = sc.broadcast(b)\n\n    total_cost = RDD.map(lambda x: (x[1] * np.log(sigmoid(linear_combination(x[0], bc_w.value, bc_b.value))) + (1 - x[1]) * np.log(1 - sigmoid(linear_combination(x[0], bc_w.value, bc_b.value)))\n    )).reduce(lambda a, b: a + b)\n    reg_cost = (lambda_reg / (2 * 11)) * np.sum(np.square(w))\n    cost = (- total_cost / m) + reg_cost\n\n    #bc_w.unpersist()\n    #bc_b.unpersist()\n    \n    return cost\n\ndef compute_gradients(record, weights, bias):\n    xi, yi = record\n    y_hat = sigmoid(linear_combination(xi, weights, bias))\n    error = y_hat - yi\n    dw = np.array([error * xi[j] for j in range(len(weights))])\n    db = error\n    return dw, db\n\ndef train(RDD_Xy, iterations, learning_rate,

In [8]:
import numpy as np
from pyspark import SparkContext

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def linear_combination(features, weights, bias):
    return sum(weights[j] * features[j] for j in range(len(weights))) + bias

def compute_gradients_and_cost(record, weights, bias):
    xi, yi = record
    y_hat = sigmoid(linear_combination(xi, weights, bias))
    error = y_hat - yi
    dw = np.zeros(len(weights))
    for j in range(len(weights)):
        dw[j] = error * xi[j]
    db = error
    cost_contribution = -yi * np.log(y_hat) - (1 - yi) * np.log(1 - y_hat)
    return (dw, db, cost_contribution)

def train(RDD_Xy, iterations, learning_rate, lambda_reg):
    np.random.seed(0) 
    feature_count = len(RDD_Xy.first()[0])
    w = np.random.rand(feature_count)
    b = np.random.rand()
    m = RDD_Xy.count()

    for n in range(iterations):
        bc_w = sc.broadcast(w)
        bc_b = sc.broadcast(b)

        gradients_and_cost = RDD_Xy.map(lambda x: compute_gradients_and_cost(x, bc_w.value, bc_b.value))
        dw, db, total_cost = gradients_and_cost.reduce(lambda a, b: (a[0] + b[0], a[1] + b[1], a[2] + b[2]))
        
        reg_cost = (lambda_reg / (2 * feature_count)) * np.sum(np.square(w))
        cost = (total_cost / m) + reg_cost
        
        # Update weights and bias
        w -= learning_rate * ((dw / m) + (lambda_reg/m) * w)
        b -= learning_rate * db / m


        print(f"Iteration {n+1}/{iterations} - Cost: {cost}")

        bc_w.unpersist()
        bc_b.unpersist()

    return w, b


In [9]:

# Now use this smaller RDD for training
w, b = train(normalized_RDD, 10, 1.5, 0)


Iteration 1/10 - Cost: 1.4998030671352984


Iteration 2/10 - Cost: 0.7452491273041107


Iteration 3/10 - Cost: 0.4463838653243955


Iteration 4/10 - Cost: 0.33598210709531867


Iteration 5/10 - Cost: 0.2858186861976214


Iteration 6/10 - Cost: 0.2581741737711223


Iteration 7/10 - Cost: 0.24076133573757508


Iteration 8/10 - Cost: 0.22876056610476408


Iteration 9/10 - Cost: 0.21995387544017125
Iteration 10/10 - Cost: 0.21318893422096608


In [10]:
print(w[0])
print(b)
print(w.shape)



-0.19840144016729716
0.016435331016430403
(11,)


In [11]:
def predict(w, b, x):
    z = 0
    for j in range(len(w)):
        z += w[j] * x[j]

    z+= b
    # Compute the sigmoid of z
    y_hat = sigmoid(z)

    if y_hat > 0.5:
        return 1
    else: 
        return 0




In [12]:
def accuracy(w, b, RDD_Xy):
    
    results = RDD_Xy.map(lambda r: 1 if predict(w, b, r[0]) == r[1] else 0)
    results = results.reduce( lambda a, b: a + b)

    accuracy = results / RDD_Xy.count()
    return accuracy




In [13]:
acc = accuracy(w, b, normalized_RDD)
print("Accuracy: ",acc )

Accuracy:  0.930181
